In [1]:
from PIL import Image
from numpy import array
import sift
from pylab import *
import os
import csv
from sklearn.cluster import KMeans

In [15]:
f = open('kmean_center.csv','r')
k_center = list()
for row in csv.reader(f):
    #data_arr = np.array([float(i) for i in row])
    data_arr = [float(i) for i in row]
    k_center.append(data_arr)
f.close()
k_center = np.array(k_center)

kmeans = KMeans(n_clusters=128, random_state=0,init=k_center,n_init=1).fit(k_center)

In [11]:
stop_word_list = list()
f = open("stop_word_list.csv",'r')
for index in csv.reader(f):
    stop_word_list.append(int(index[0]))
f.close
print stop_word_list

[60, 74, 21, 76, 125, 22, 25, 95, 52, 47]


In [17]:
DATASET_SIFT ='dataset_sift_update'
sift_data = list()

for filename in os.listdir(DATASET_SIFT):
    
    f = loadtxt(DATASET_SIFT+'/'+filename)

    if len(f.shape) <= 1:
        continue
        
    l1,d1 = sift.read_features_from_file(DATASET_SIFT+'/'+filename)
    
    img_vector = kmeans.predict(d1)
    x = [0]*128
    for num in img_vector:
        if not(num in stop_word_list):
            x[num] = x[num]+1
    img_row = [filename,','.join(str(i) for i in x)]
    sift_data.append(img_row)
    f = open("sift_update_with_stop_word.csv","w")
    w = csv.writer(f)
    w.writerows(sift_data)
    f.close()
        

In [20]:
def vector_distance(arr1,arr2):
    if len(arr1) != len(arr2):
        return None
    sum,sum_arr1,sum_arr2 = 0,0,0
    
    for i in xrange(len(arr1)):
        sum += arr1[i]*arr2[i]
        sum_arr1 += arr1[i]**2
        sum_arr2 += arr2[i]**2
    
    sum_arr1 = sum_arr1**0.5
    sum_arr2 = sum_arr2**0.5
        
    return sum/(sum_arr1*sum_arr2)

In [23]:
q_img ='dataset_update/ukbench00000.jpg'
sift.process_image(q_img,'q_img.sift')

l1,d1 = sift.read_features_from_file('q_img.sift')
q_img_vector = kmeans.predict(d1)

q_vector = [0]*128
for num in q_img_vector:
    if not(num in stop_word_list):
        q_vector[num] = q_vector[num]+1

f = open('sift_update_with_stop_word.csv','r')
rank = list()
for row in csv.reader(f):
    distance = vector_distance(q_vector,[int(i) for i in row[1].split(',')])
    
    if distance == None:
        continue
        
    rank_data = (row[0],distance)
    rank.append(rank_data)
f.close()

rank = sorted(rank,key=lambda rank:rank[1])
rank = rank[::-1]
print rank[:10]



processed tmp.pgm to q_img.sift
[('ukbench00000.sift', 0.9999999999999999), ('ukbench00660.sift', 0.6155607992531347), ('ukbench00154.sift', 0.6060979624294756), ('ukbench00621.sift', 0.5904437765029024), ('ukbench00015.sift', 0.589625420217025), ('ukbench00493.sift', 0.5883790037787072), ('ukbench00209.sift', 0.5865464152543313), ('ukbench00879.sift', 0.5782430136778557), ('ukbench00310.sift', 0.5711972394287688), ('ukbench00825.sift', 0.5689117365027191)]
